# Custom Imputer Approach

I tried to impute the data with mean, mod, and more but then I realized the fact that a custom imputer will do that better than me. So I decided to do so. I will be using LGBMImputer for imputing the missing values.

In [26]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import scipy as sc
import seaborn as sbn

import warnings
warnings.filterwarnings("ignore")


dff=pd.read_csv('../input/spaceship-titanic/train.csv')
df_test=pd.read_csv('../input/spaceship-titanic/test.csv')

dff

,PassengerId,HomePlanet,CryoSleep,Cabin,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name,Transported
0,0001_01,Europa,False,B/0/P,TRAPPIST-1e,39.0,False,0.0,0.0,0.0,0.0,0.0,Maham Ofracculy,False
1,0002_01,Earth,False,F/0/S,TRAPPIST-1e,24.0,False,109.0,9.0,25.0,549.0,44.0,Juanna Vines,True
2,0003_01,Europa,False,A/0/S,TRAPPIST-1e,58.0,True,43.0,3576.0,0.0,6715.0,49.0,Altark Susent,False
3,0003_02,Europa,False,A/0/S,TRAPPIST-1e,33.0,False,0.0,1283.0,371.0,3329.0,193.0,Solam Susent,False
4,0004_01,Earth,False,F/1/S,TRAPPIST-1e,16.0,False,303.0,70.0,151.0,565.0,2.0,Willy Santantines,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8688,9276_01,Europa,False,A/98/P,55 Cancri e,41.0,True,0.0,6819.0,0.0,1643.0,74.0,Gravior Noxnuther,False
8689,9278_01,Earth,True,G/1499/S,PSO J318.5-22,18.0,False,0.0,0.0,0.0,0.0,0.0,Kurta Mondalley,False
8690,9279_01,Earth,False,G/1500/S,TRAPPIST-1e,26.0,False,0.0,0.0,1872.0,1.0,0.0,Fayey Connon,True
8691,9280_01,Europa,False,E/608/S,55 Cancri e,32.0,False,0.0,1049.0,0.0,353.0,3235.0,Celeon Hontichre,False


In [27]:
dff.corr()

,Age,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
Age,1.000000,0.068723,0.130421,0.033133,0.123970,0.101007,-0.075026
RoomService,0.068723,1.000000,-0.015889,0.054480,0.010080,-0.019581,-0.244611
FoodCourt,0.130421,-0.015889,1.000000,-0.014228,0.221891,0.227995,0.046566
ShoppingMall,0.033133,0.054480,-0.014228,1.000000,0.013879,-0.007322,0.010141
Spa,0.123970,0.010080,0.221891,0.013879,1.000000,0.153821,-0.221131
VRDeck,0.101007,-0.019581,0.227995,-0.007322,0.153821,1.000000,-0.207075
Transported,-0.075026,-0.244611,0.046566,0.010141,-0.221131,-0.207075,1.000000


I am dropping the cabin because it is hard to preprocess and when I preprocessed it, it didn't make any difference...

In [28]:
for df in [dff,df_test]:
    #df["Cabin"]= df["Cabin"].str.split("/", n = 1, expand = True)[0]
    df.drop('Cabin',axis=1,inplace=True)
df

,PassengerId,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Name
0,0013_01,Earth,True,TRAPPIST-1e,27.0,False,0.0,0.0,0.0,0.0,0.0,Nelly Carsoning
1,0018_01,Earth,False,TRAPPIST-1e,19.0,False,0.0,9.0,0.0,2823.0,0.0,Lerome Peckers
2,0019_01,Europa,True,55 Cancri e,31.0,False,0.0,0.0,0.0,0.0,0.0,Sabih Unhearfus
3,0021_01,Europa,False,TRAPPIST-1e,38.0,False,0.0,6652.0,0.0,181.0,585.0,Meratz Caltilter
4,0023_01,Earth,False,TRAPPIST-1e,20.0,False,10.0,0.0,635.0,0.0,0.0,Brence Harperez
...,...,...,...,...,...,...,...,...,...,...,...,...
4272,9266_02,Earth,True,TRAPPIST-1e,34.0,False,0.0,0.0,0.0,0.0,0.0,Jeron Peter
4273,9269_01,Earth,False,TRAPPIST-1e,42.0,False,0.0,847.0,17.0,10.0,144.0,Matty Scheron
4274,9271_01,Mars,True,55 Cancri e,NaN,False,0.0,0.0,0.0,0.0,0.0,Jayrin Pore
4275,9273_01,Europa,False,NaN,NaN,False,0.0,2680.0,0.0,0.0,523.0,Kitakan Conale


In [29]:
for df in [dff,df_test]:
    df.drop(['Name','PassengerId'],axis=1,inplace=True)
    
    
    
    for i in ['CryoSleep','VIP','Transported']:
        try:
            df[i]=pd.get_dummies(df[i])[1]
        except:
            continue

    
    for i in ['HomePlanet','Destination']:
        df[i].fillna('Missing',inplace=True)
        df[i]=df[i].astype('category').cat.codes
    
    #for i in ['RoomService','FoodCourt','ShoppingMall','Spa','VRDeck']:
    #    df[i].fillna(0,inplace=True)
        

In [30]:
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold, StratifiedKFold
import lightgbm as lgb
from catboost import CatBoostRegressor
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sbn
IMPUTE = True

I am adjusting our custom imputer settings.

In [31]:
def analyze_column(input_series: pd.Series) -> str:
    if pd.api.types.is_numeric_dtype(input_series):
        return 'numeric'
    else:
        return 'categorical'
    


class LGBMImputer:
    '''
    Regression imputer using LightGBM
    '''
    def __init__(self, cat_features=[], n_iter=15000, verbose=False):
        self.n_iter = n_iter
        self.cat_features = cat_features
        self.verbose = verbose
        self.n_features = None
        self.feature_names = None
        self.feature_with_missing = None
        self.imputers = {}
        self.offsets = {}
        self.objectives = {} 
                                                                            # Özellikler-Parametreler belirleniyor...
        
    def fit_transform(self, X, y=None):
        output_X = X.copy()
        self.n_features = X.shape[1]

        if isinstance(X, pd.DataFrame):
            self.feature_names = X.columns.tolist()
        else:
            self.feature_names = [f'f{i}' for i in range(self.n_features)]
            X = pd.DataFrame(X, columns=self.feature_names)
        self.feature_with_missing = [col for col in self.feature_names if X[col].isnull().sum() > 0]

        for icol, col in enumerate(self.feature_with_missing):              #icol sıra numarası, col kolon
            if icol in self.cat_features:
                nuni = X[col].dropna().nunique()
                if nuni == 2:
                    params = {
                        'objective': 'binary'
                    }
                elif nuni > 2:
                    params = {
                        'objective': 'multiclass',
                        'num_class': nuni + 1
                    }
            else: # automatic analyze column
                if analyze_column(X[col]) == 'numeric':
                    params = {
                        'objective': 'regression'
                    }
                else:
                    nuni = X[col].dropna().nunique()
                    if nuni == 2:
                        params = {
                            'objective': 'binary'
                        }
                    elif nuni > 2:
                        params = {
                            'objective': 'multiclass',
                            'num_class': nuni + 1
                        }
                    else:
                        print(f'column {col} has only one unique value.')
                        continue

            params['verbosity'] = -1
            
            null_idx = X[col].isnull()
            x_train = X.loc[~null_idx].drop(col, axis=1)
            x_test = X.loc[null_idx].drop(col, axis=1)
            y_offset = X[col].min()
            y_train = X.loc[~null_idx, col].astype(int) - y_offset
            dtrain = lgb.Dataset(
                data=x_train,
                label=y_train
            )

            early_stopping_rounds = 50
            model = lgb.train(
                params, dtrain, valid_sets=[dtrain], 
                num_boost_round=self.n_iter,
                early_stopping_rounds=early_stopping_rounds,
                verbose_eval=0,
            )

            y_test = model.predict(x_test)
            if params['objective'] == 'multiclass':
                y_test = np.argmax(y_test, axis=1).astype(float)
            elif params['objective'] == 'binary':
                y_test = (y_test > 0.5).astype(float)
            y_test += y_offset
            output_X.loc[null_idx, col] = y_test
            if params['objective'] in ['multiclass', 'binary']:
                output_X[col] = output_X[col].astype(int)
            self.imputers[col] = model
            self.offsets[col] = y_offset
            self.objectives[col] = params['objective']
            if self.verbose:
                print(f'{col}:\t{self.objectives[col]}...iter{model.best_iteration}/{self.n_iter}')
        
        return output_X    

In [32]:
if IMPUTE:
    s=0
    for df in [dff,df_test]:
        feature_df = df
        
        feature_df = feature_df.replace(99999.0,np.nan)

        imputer = LGBMImputer(verbose=True)
        feature_df.loc[:, feature_df.columns[1:]] = imputer.fit_transform(feature_df[feature_df.columns[1:]])

        if s==0:
            train_df = feature_df.copy().reset_index(drop=True)
            s=1
        else:
            test_df = feature_df.copy().reset_index(drop=True)

Age:	regression...iter15000/15000
RoomService:	regression...iter15000/15000
FoodCourt:	regression...iter15000/15000
ShoppingMall:	regression...iter15000/15000
Spa:	regression...iter15000/15000
VRDeck:	regression...iter15000/15000
Age:	regression...iter15000/15000
RoomService:	regression...iter15000/15000
FoodCourt:	regression...iter15000/15000
ShoppingMall:	regression...iter15000/15000
Spa:	regression...iter15000/15000
VRDeck:	regression...iter15000/15000


I am changing the variable names and checking their correlation to get better model performance with high correlated variables.

In [33]:
df=train_df
df_test=test_df

In [34]:
df.corr()

,HomePlanet,CryoSleep,Destination,Age,VIP,RoomService,FoodCourt,ShoppingMall,Spa,VRDeck,Transported
HomePlanet,1.000000,0.080988,0.021099,0.122964,0.114819,0.190545,0.055776,0.088528,0.042546,0.034383,0.107755
CryoSleep,0.080988,1.000000,-0.080452,-0.072178,-0.078281,-0.248160,-0.208807,-0.211827,-0.201507,-0.195235,0.460132
Destination,0.021099,-0.080452,1.000000,-0.014617,-0.042404,0.041858,-0.117732,0.021926,-0.064232,-0.078426,-0.109179
Age,0.122964,-0.072178,-0.014617,1.000000,0.090730,0.068568,0.130952,0.034725,0.125407,0.101349,-0.075828
VIP,0.114819,-0.078281,-0.042404,0.090730,1.000000,0.058239,0.130777,0.022468,0.060520,0.122275,-0.037261
RoomService,0.190545,-0.248160,0.041858,0.068568,0.058239,1.000000,-0.016886,0.055167,0.008457,-0.022193,-0.246935
FoodCourt,0.055776,-0.208807,-0.117732,0.130952,0.130777,-0.016886,1.000000,-0.011146,0.221870,0.227002,0.047115
ShoppingMall,0.088528,-0.211827,0.021926,0.034725,0.022468,0.055167,-0.011146,1.000000,0.023253,-0.006027,0.007488
Spa,0.042546,-0.201507,-0.064232,0.125407,0.060520,0.008457,0.221870,0.023253,1.000000,0.148920,-0.221510
VRDeck,0.034383,-0.195235,-0.078426,0.101349,0.122275,-0.022193,0.227002,-0.006027,0.148920,1.000000,-0.207985


In [35]:
df.drop(['ShoppingMall','VIP'],axis=1,inplace=True)
df_test.drop(['ShoppingMall','VIP'],axis=1,inplace=True)


In [36]:
X = df.drop("Transported", axis=1)
Y = df.Transported

sub = df_test.copy()

In [37]:
from sklearn.model_selection import train_test_split

X_train, X_val, y_train, y_val = train_test_split(X, Y, test_size=0.33, random_state=42)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()

X_train_scaled = scaler.fit_transform(X_train)

X_val_scaled = scaler.fit_transform(X_val)
sub = scaler.transform(sub)

I have tried bunch of models to get better results. If you want to try it yourself, feel free to copy and edit...

In [38]:
from sklearn.model_selection import cross_validate, GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.naive_bayes import GaussianNB
from xgboost import XGBClassifier
from lightgbm import LGBMClassifier
from sklearn.metrics import classification_report, accuracy_score, f1_score, confusion_matrix
from sklearn.model_selection import train_test_split
from catboost import CatBoostClassifier


I got 0.7884 with CatBoost but I want to push even more...

In [ ]:
#Score 0.7884
clf=CatBoostClassifier()
clf.fit(X_train_scaled,y_train,eval_set=(X_val_scaled,y_val))

In [ ]:
svm = LGBMClassifier(objective='binary')
params = {
    'n_estimators' :(50,75,100),
    'learning_rate' : (0.08,0.01)
}
clf = GridSearchCV(svm, params, cv=10)
clf.fit(X_train_scaled, y_train,eval_set=(X_val_scaled,y_val));
print("Best hyperparameter:", clf.best_params_)

In [41]:
y_pred = clf.predict(sub)

y_pred= pd.DataFrame(y_pred)
y_pred.replace({0: False, 1: True}, inplace=True)
submission = pd.read_csv("../input/spaceship-titanic/sample_submission.csv")
submission["Transported"] = y_pred
submission.to_csv('submission.csv', index=False)